<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-Property" data-toc-modified-id="Read-Property-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read Property</a></span></li><li><span><a href="#Distance-Bus-Stop" data-toc-modified-id="Distance-Bus-Stop-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Distance Bus Stop</a></span></li><li><span><a href="#Distance-Train-Station" data-toc-modified-id="Distance-Train-Station-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Distance Train Station</a></span></li><li><span><a href="#Distance-School" data-toc-modified-id="Distance-School-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Distance School</a></span></li><li><span><a href="#Calculate-Routine-Distance-by-openrouteservice" data-toc-modified-id="Calculate-Routine-Distance-by-openrouteservice-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Calculate Routine Distance by openrouteservice</a></span></li></ul></div>

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Read Property

In [2]:
df_property=pd.read_csv("../data/curated/domain_2.csv")


df_property=df_property[["url","coordinates"]].copy()
df_property["longitude"]=df_property["coordinates"].map(lambda x: eval(x)[1])
df_property["latitude"]=df_property["coordinates"].map(lambda x: eval(x)[0])

# create gdf
gdf_property = gpd.GeoDataFrame(df_property, geometry=gpd.points_from_xy(df_property.longitude, df_property.latitude, crs="EPSG:4326"))
gdf_property['geometry']  = gdf_property['geometry'] .to_crs("EPSG:4326")
gdf_property=gdf_property[["url","geometry"]].copy()
gdf_property=gdf_property.reset_index(drop=True)
gdf_property.head()

,url,geometry
0,https://www.domain.com.au/1202-620-collins-str...,POINT (144.95435 -37.81866)
1,https://www.domain.com.au/1701-620-collins-str...,POINT (144.95435 -37.81866)
2,https://www.domain.com.au/5707b-639-little-lon...,POINT (144.95217 -37.81419)
3,https://www.domain.com.au/1003-225-elizabeth-s...,POINT (144.96232 -37.81323)
4,https://www.domain.com.au/1202-31-spring-stree...,POINT (144.97407 -37.81450)


In [3]:
gdf_property.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

# Distance Bus Stop

In [4]:
gdf_busstop=gpd.read_file("../data/raw/PTV_data/PTV_METRO_BUS_STOP/PTV_METRO_BUS_STOP.shp")
gdf_busstop=gdf_busstop[['STOP_ID', 'STOP_NAME', 'LATITUDE', 'LONGITUDE', 'geometry']].copy()
gdf_busstop['geometry']  = gdf_busstop['geometry'] .to_crs("EPSG:4326")

gdf_regionalbusstop=gpd.read_file("../data/raw/PTV_data/PTV_REGIONAL_BUS_STOP/PTV_REGIONAL_BUS_STOP.shp")
gdf_regionalbusstop=gdf_regionalbusstop[['STOP_ID', 'STOP_NAME', 'LATITUDE', 'LONGITUDE',  'geometry']].copy()
gdf_regionalbusstop['geometry']  = gdf_regionalbusstop['geometry'] .to_crs("EPSG:4326")

gdf_busstop=pd.concat([gdf_busstop,gdf_regionalbusstop],ignore_index=True)
gdf_busstop.head()

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,geometry
0,10185,Bonnie View Rd/Maroondah Hwy (Croydon North),-37.780920,145.288145,POINT (145.28815 -37.78091)
1,10186,Primrose Rd/Maroondah Hwy (Croydon North),-37.774626,145.292843,POINT (145.29285 -37.77461)
2,10187,Exeter Rd/Maroondah Hwy (Croydon North),-37.770446,145.295459,POINT (145.29546 -37.77043)
3,10188,Maroondah Golf Park/Maroondah Hwy (Chirnside P...,-37.766346,145.301775,POINT (145.30178 -37.76633)
4,10189,Old Melbourne Rd/Maroondah Hwy (Chirnside Park),-37.765276,145.304410,POINT (145.30442 -37.76526)


In [5]:
gdf_busstop.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
# define function to calculate the top20 nearest locations' coordinates by geopandas

def get_20_nearest_locations(gdf, point):
    dist_series=gdf["geometry"].distance(point)
    top10_idx=dist_series.argsort().values[:20]
    
    top20_cordinates=gdf.iloc[top10_idx]["geometry"].map(lambda p:(p.x,p.y)).values

    return top20_cordinates

get_20_nearest_locations(gdf=gdf_busstop, point=gdf_property["geometry"][1])

array([(144.95351489772105, -37.819311816789444),
       (144.9534158977541, -37.819165836781785),
       (144.95512985798737, -37.81427487686854),
       (144.95208281845862, -37.81480785666372),
       (144.95530301796674, -37.81418784687983),
       (144.95581277787238, -37.81424184691499),
       (144.9511550985912, -37.815070836602),
       (144.95702705770026, -37.81388481699575),
       (144.94912109836338, -37.8204248464962),
       (144.94903289830071, -37.82114385649469),
       (144.95787989760046, -37.81343985705128),
       (144.9610658963954, -37.81944285730619),
       (144.9611098167439, -37.81616280728887),
       (144.960704096913, -37.815254797255506),
       (144.9613020566593, -37.81663485730496),
       (144.95922485738694, -37.813255807142035),
       (144.96037181705063, -37.81451886722826),
       (144.96032789706518, -37.814454877224826),
       (144.96118109678523, -37.81566879729057),
       (144.96180605631926, -37.818959827353794)], dtype=object)

In [7]:
gdf_property["top20_near_busstop"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_busstop, point=x))
gdf_property.head()

,url,geometry,top20_near_busstop
0,https://www.domain.com.au/1202-620-collins-str...,POINT (144.95435 -37.81866),"[(144.95351489772105, -37.819311816789444), (1..."
1,https://www.domain.com.au/1701-620-collins-str...,POINT (144.95435 -37.81866),"[(144.95351489772105, -37.819311816789444), (1..."
2,https://www.domain.com.au/5707b-639-little-lon...,POINT (144.95217 -37.81419),"[(144.95208281845862, -37.81480785666372), (14..."
3,https://www.domain.com.au/1003-225-elizabeth-s...,POINT (144.96232 -37.81323),"[(144.96178985702383, -37.81249782731262), (14..."
4,https://www.domain.com.au/1202-31-spring-stree...,POINT (144.97407 -37.81450),"[(144.97132877567088, -37.80970683794742), (14..."


# Distance Train Station

In [8]:
gdf_trainstation=gpd.read_file("../data/raw/PTV_data/PTV_METRO_TRAIN_STATION/PTV_METRO_TRAIN_STATION.shp")
gdf_trainstation['geometry']  = gdf_trainstation['geometry'] .to_crs("EPSG:4326")
gdf_trainstation.head()

,STOP_ID,STOP_NAME,LATITUDE,LONGITUDE,TICKETZONE,ROUTEUSSP,geometry
0,19970,Royal Park Railway Station (Parkville),-37.781193,144.952301,1,Upfield,POINT (144.95231 -37.78118)
1,19971,Flemington Bridge Railway Station (North Melbo...,-37.788140,144.939323,1,Upfield,POINT (144.93933 -37.78813)
2,19972,Macaulay Railway Station (North Melbourne),-37.794267,144.936166,1,Upfield,POINT (144.93617 -37.79425)
3,19973,North Melbourne Railway Station (West Melbourne),-37.807419,144.942570,1,"Flemington,Sunbury,Upfield,Werribee,Williamsto...",POINT (144.94258 -37.80741)
4,19974,Clifton Hill Railway Station (Clifton Hill),-37.788657,144.995417,1,"Mernda,Hurstbridge",POINT (144.99542 -37.78864)


In [9]:
gdf_property["top20_near_trainstation"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_trainstation, point=x))
gdf_property.head()

,url,geometry,top20_near_busstop,top20_near_trainstation
0,https://www.domain.com.au/1202-620-collins-str...,POINT (144.95435 -37.81866),"[(144.95351489772105, -37.819311816789444), (1...","[(144.9514168182362, -37.817922846637515), (14..."
1,https://www.domain.com.au/1701-620-collins-str...,POINT (144.95435 -37.81866),"[(144.95351489772105, -37.819311816789444), (1...","[(144.9514168182362, -37.817922846637515), (14..."
2,https://www.domain.com.au/5707b-639-little-lon...,POINT (144.95217 -37.81419),"[(144.95208281845862, -37.81480785666372), (14...","[(144.9514168182362, -37.817922846637515), (14..."
3,https://www.domain.com.au/1003-225-elizabeth-s...,POINT (144.96232 -37.81323),"[(144.96178985702383, -37.81249782731262), (14...","[(144.96259985716247, -37.80992580735206), (14..."
4,https://www.domain.com.au/1202-31-spring-stree...,POINT (144.97407 -37.81450),"[(144.97132877567088, -37.80970683794742), (14...","[(144.97291709525015, -37.811040818064356), (1..."


# Distance School

+ there are 4 types of schools, we calculate the distance by school type

In [11]:
df_school = pd.read_excel("../data/raw/SearchResults.xlsx")
df_school.head()

,ACARA ID,School Name,Suburb,State,Postcode,Type,Sector,Status,Geolocation,Parent School ID,AGE ID,Latitude,Longitude
0,44370,Abbotsford Primary School,Abbotsford,VIC,3067,Primary,Gov,Open,Major Cities,44370,48050.0,-37.8091,144.999
1,44727,Aberfeldie Primary School,Essendon,VIC,3040,Primary,Gov,Open,Major Cities,44727,44744.0,-37.7566,144.896
2,45704,Academy of Mary Immaculate,Fitzroy,VIC,3065,Secondary,Non-Gov,Open,Major Cities,45704,40739.0,-37.8037,144.974
3,46213,Adass Israel School,Elsternwick,VIC,3185,Combined,Non-Gov,Open,Major Cities,46213,40740.0,-37.8839,145.008
4,52146,Adass Israel School - Glen Eira Road Campus,Elsternwick,VIC,3185,Combined,Non-Gov,Open,Major Cities,46213,NaN,-37.8769,144.998


In [12]:
df_school["Type"].value_counts()

Primary      1646
Secondary     548
Combined      404
Special       198
Name: Type, dtype: int64

In [13]:
# create gdf
gdf_school = gpd.GeoDataFrame(df_school, geometry=gpd.points_from_xy(df_school.Longitude, df_school.Latitude, crs="EPSG:4326"))
gdf_school['geometry']  = gdf_school['geometry'] .to_crs("EPSG:4326")
gdf_school.head()

,ACARA ID,School Name,Suburb,State,Postcode,Type,Sector,Status,Geolocation,Parent School ID,AGE ID,Latitude,Longitude,geometry
0,44370,Abbotsford Primary School,Abbotsford,VIC,3067,Primary,Gov,Open,Major Cities,44370,48050.0,-37.8091,144.999,POINT (144.99900 -37.80910)
1,44727,Aberfeldie Primary School,Essendon,VIC,3040,Primary,Gov,Open,Major Cities,44727,44744.0,-37.7566,144.896,POINT (144.89600 -37.75660)
2,45704,Academy of Mary Immaculate,Fitzroy,VIC,3065,Secondary,Non-Gov,Open,Major Cities,45704,40739.0,-37.8037,144.974,POINT (144.97400 -37.80370)
3,46213,Adass Israel School,Elsternwick,VIC,3185,Combined,Non-Gov,Open,Major Cities,46213,40740.0,-37.8839,145.008,POINT (145.00800 -37.88390)
4,52146,Adass Israel School - Glen Eira Road Campus,Elsternwick,VIC,3185,Combined,Non-Gov,Open,Major Cities,46213,NaN,-37.8769,144.998,POINT (144.99800 -37.87690)


In [14]:
# primary school
gdf_property["top20_near_primary"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_school[gdf_school["Type"]=='Primary'], point=x))

# Secondary school
gdf_property["top20_near_secondary"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_school[gdf_school["Type"]=='Secondary'], point=x))


# Combined school
gdf_property["top20_near_combined"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_school[gdf_school["Type"]=='Combined'], point=x))


# special school
gdf_property["top20_near_special"]=gdf_property["geometry"].map(lambda x: get_20_nearest_locations(gdf=gdf_school[gdf_school["Type"]=='Special'], point=x))
gdf_property.head()

,url,geometry,top20_near_busstop,top20_near_trainstation,top20_near_primary,top20_near_secondary,top20_near_combined,top20_near_special
0,https://www.domain.com.au/1202-620-collins-str...,POINT (144.95435 -37.81866),"[(144.95351489772105, -37.819311816789444), (1...","[(144.9514168182362, -37.817922846637515), (14...","[(144.954, -37.8292), (144.94181, -37.81244), ...","[(144.955, -37.8128), (144.955, -37.8128), (14...","[(144.956, -37.8172), (144.953, -37.8111), (14...","[(144.957, -37.8199), (144.95, -37.8306), (144..."
1,https://www.domain.com.au/1701-620-collins-str...,POINT (144.95435 -37.81866),"[(144.95351489772105, -37.819311816789444), (1...","[(144.9514168182362, -37.817922846637515), (14...","[(144.954, -37.8292), (144.94181, -37.81244), ...","[(144.955, -37.8128), (144.955, -37.8128), (14...","[(144.956, -37.8172), (144.953, -37.8111), (14...","[(144.957, -37.8199), (144.95, -37.8306), (144..."
2,https://www.domain.com.au/5707b-639-little-lon...,POINT (144.95217 -37.81419),"[(144.95208281845862, -37.81480785666372), (14...","[(144.9514168182362, -37.817922846637515), (14...","[(144.94181, -37.81244), (144.954, -37.8292), ...","[(144.955, -37.8128), (144.955, -37.8128), (14...","[(144.953, -37.8111), (144.953, -37.8111), (14...","[(144.957, -37.8199), (144.955, -37.8051), (14..."
3,https://www.domain.com.au/1003-225-elizabeth-s...,POINT (144.96232 -37.81323),"[(144.96178985702383, -37.81249782731262), (14...","[(144.96259985716247, -37.80992580735206), (14...","[(144.97, -37.802), (144.954, -37.8292), (144....","[(144.955, -37.8128), (144.955, -37.8128), (14...","[(144.961, -37.8126), (144.956, -37.8172), (14...","[(144.957, -37.8199), (144.955, -37.8051), (14..."
4,https://www.domain.com.au/1202-31-spring-stree...,POINT (144.97407 -37.81450),"[(144.97132877567088, -37.80970683794742), (14...","[(144.97291709525015, -37.811040818064356), (1...","[(144.97, -37.802), (144.979, -37.802), (144.9...","[(144.971, -37.8115), (144.972, -37.8103), (14...","[(144.961, -37.8126), (144.986, -37.8024), (14...","[(144.985, -37.8078), (144.957, -37.8199), (14..."


In [15]:
gdf_property["property_cordinate"]=gdf_property["geometry"].map(lambda p:(p.x,p.y))
gdf_property.head()

,url,geometry,top20_near_busstop,top20_near_trainstation,top20_near_primary,top20_near_secondary,top20_near_combined,top20_near_special,property_cordinate
0,https://www.domain.com.au/1202-620-collins-str...,POINT (144.95435 -37.81866),"[(144.95351489772105, -37.819311816789444), (1...","[(144.9514168182362, -37.817922846637515), (14...","[(144.954, -37.8292), (144.94181, -37.81244), ...","[(144.955, -37.8128), (144.955, -37.8128), (14...","[(144.956, -37.8172), (144.953, -37.8111), (14...","[(144.957, -37.8199), (144.95, -37.8306), (144...","(144.9543471, -37.8186626)"
1,https://www.domain.com.au/1701-620-collins-str...,POINT (144.95435 -37.81866),"[(144.95351489772105, -37.819311816789444), (1...","[(144.9514168182362, -37.817922846637515), (14...","[(144.954, -37.8292), (144.94181, -37.81244), ...","[(144.955, -37.8128), (144.955, -37.8128), (14...","[(144.956, -37.8172), (144.953, -37.8111), (14...","[(144.957, -37.8199), (144.95, -37.8306), (144...","(144.9543471, -37.8186626)"
2,https://www.domain.com.au/5707b-639-little-lon...,POINT (144.95217 -37.81419),"[(144.95208281845862, -37.81480785666372), (14...","[(144.9514168182362, -37.817922846637515), (14...","[(144.94181, -37.81244), (144.954, -37.8292), ...","[(144.955, -37.8128), (144.955, -37.8128), (14...","[(144.953, -37.8111), (144.953, -37.8111), (14...","[(144.957, -37.8199), (144.955, -37.8051), (14...","(144.9521662, -37.8141942)"
3,https://www.domain.com.au/1003-225-elizabeth-s...,POINT (144.96232 -37.81323),"[(144.96178985702383, -37.81249782731262), (14...","[(144.96259985716247, -37.80992580735206), (14...","[(144.97, -37.802), (144.954, -37.8292), (144....","[(144.955, -37.8128), (144.955, -37.8128), (14...","[(144.961, -37.8126), (144.956, -37.8172), (14...","[(144.957, -37.8199), (144.955, -37.8051), (14...","(144.9623173, -37.8132308)"
4,https://www.domain.com.au/1202-31-spring-stree...,POINT (144.97407 -37.81450),"[(144.97132877567088, -37.80970683794742), (14...","[(144.97291709525015, -37.811040818064356), (1...","[(144.97, -37.802), (144.979, -37.802), (144.9...","[(144.971, -37.8115), (144.972, -37.8103), (14...","[(144.961, -37.8126), (144.986, -37.8024), (14...","[(144.985, -37.8078), (144.957, -37.8199), (14...","(144.9740695, -37.8145047)"


# Calculate Routine Distance by openrouteservice

In [16]:
import openrouteservice as ors

def calculate_route_distance(start_cordinate, end_cordinates):
    client = ors.Client(base_url="http://localhost:8080/ors")
   
    distances=[]
    
    # calculate 20 times, as we have 20 coordinates for each [busstop, trainstation...]
    for i in range(20):
        coordinates = [[start_cordinate[0], start_cordinate[1]],[end_cordinates[i][0], end_cordinates[i][1]]]
        
        try:
            route = client.directions(
                coordinates=coordinates,
                profile='driving-car',
                format='geojson',
                validate=False,
            )
            distances.append(route['features'][0]["properties"]["summary"]["distance"])
        except Exception as err:
            
            # if error, the distance is more than 100000 metres
            distances.append(1000000)
#             print(err)
        
    # 
    return np.nanmin(np.array(distances))

In [17]:
%%time

gdf_property["dist_nearest_busstop"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_busstop"]), axis=1)

gdf_property["dist_nearest_trainstation"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_trainstation"]), axis=1)


gdf_property["dist_nearest_primary_school"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_primary"]), axis=1)

gdf_property["dist_nearest_secondary_school"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_secondary"]), axis=1)

gdf_property["dist_nearest_combined_school"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_combined"]), axis=1)

gdf_property["dist_nearest_special_school"]=gdf_property.apply(lambda row: 
                                                        calculate_route_distance(
                                                            start_cordinate=row["property_cordinate"], 
                                                            end_cordinates=row["top20_near_special"]), axis=1)

CPU times: user 12min 38s, sys: 31.8 s, total: 13min 10s
Wall time: 13min 10s


In [25]:
df_property=pd.read_csv("../data/curated/domain_2.csv")

df_property_result=pd.merge(df_property, gdf_property[['url','dist_nearest_busstop', 'dist_nearest_trainstation', 
                                                       'dist_nearest_primary_school', 'dist_nearest_secondary_school', 'dist_nearest_combined_school',
                                                       'dist_nearest_special_school']], on="url",how="left")
df_property_result.head()

,url,name,cost_text,type,school_title,school_distance,suburb_sold,suburb_avg_day,suburb_population,suburb_age,...,Median_tot_fam_inc_weekly,Tot_P_P,Median_age_persons,Median_rent_weekly,dist_nearest_busstop,dist_nearest_trainstation,dist_nearest_primary_school,dist_nearest_secondary_school,dist_nearest_combined_school,dist_nearest_special_school
0,https://www.domain.com.au/1202-620-collins-str...,1202/620 Collins Street Melbourne VIC 3000,$500 per week,Apartment / Unit / Flat,University High School,2.4,596,147,"47,279",20 to 39,...,2248.0,9848.0,30.0,375.0,1000000,1000000,1000000,1000000,1000000,1000000
1,https://www.domain.com.au/1701-620-collins-str...,1701/620 Collins Street Melbourne VIC 3000,$650 per week,Apartment / Unit / Flat,University High School,2.4,596,147,"47,279",20 to 39,...,2248.0,9848.0,30.0,375.0,1000000,1000000,1000000,1000000,1000000,1000000
2,https://www.domain.com.au/5707b-639-little-lon...,5707B/639 Little Lonsdale Street Melbourne VIC...,$570,Apartment / Unit / Flat,University High School,2.4,387,120,"47,279",20 to 39,...,2248.0,9848.0,30.0,375.0,1000000,1000000,1000000,1000000,1000000,1000000
3,https://www.domain.com.au/1003-225-elizabeth-s...,1003/225 Elizabeth Street Melbourne VIC 3000,$575.00 per week,Apartment / Unit / Flat,Carlton Gardens Primary School,2.4,596,147,"47,279",20 to 39,...,2248.0,9848.0,30.0,375.0,1000000,1000000,1000000,1000000,1000000,1000000
4,https://www.domain.com.au/1202-31-spring-stree...,1202/31 Spring Street Melbourne VIC 3000,$570.00,Apartment / Unit / Flat,Carlton Gardens Primary School,2.4,596,147,"47,279",20 to 39,...,2248.0,9848.0,30.0,375.0,1000000,1000000,1000000,1000000,1000000,1000000


In [26]:
df_property_result.shape

(12767, 34)

In [27]:
df_property_result["dist_nearest_primary_school"]=df_property_result.apply(lambda row: 
                                                                           row["dist_nearest_primary_school"] 
                                                                           if row["dist_nearest_primary_school"] <row["dist_nearest_combined_school"] 
                                                                          else row["dist_nearest_combined_school"], axis=1)

df_property_result["dist_nearest_secondary_school"]=df_property_result.apply(lambda row: 
                                                                           row["dist_nearest_secondary_school"] 
                                                                           if row["dist_nearest_secondary_school"] <row["dist_nearest_combined_school"] 
                                                                          else row["dist_nearest_combined_school"], axis=1)

df_property_result=df_property_result.drop(["dist_nearest_combined_school","dist_nearest_special_school"], axis=1)
df_property_result.to_csv("../data/curated/df_domain_final2.csv", index=False)